In [76]:
import numpy
import random
from collections import defaultdict
import urllib
import math
import random
import collections
import string
import csv
import sys
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import svm
import sklearn.metrics
from sklearn.svm import LinearSVC
from nltk.corpus import stopwords

In [77]:
def parseLabeledData(path):
    with open(path, 'r') as file:
        for line in file:
            line = line.split(',')
            yield({"asin":line[0],
                 "question":line[1],
                 "review":line[2],
                 "answer":line[3].strip()
                })
        

print("Reading labeled data...")
data = list(parseLabeledData("C:/Users/Moi/Downloads/highestReviewData.csv"))
#data = parseLabeledData("/Users/Silvia/Desktop/New Data - Sheet1.csv")
queries = [d['question'] for d in data]
answers = [d['answer'] for d in data]
reviews = [d['review'] for d in data]
print("done")

Reading labeled data...
done


In [78]:
def parseAllQueries(path):
    file = open(path, 'r')
    dataList = defaultdict(lambda: [])
    for line in file:
        line = eval(line)
        dataList[line["asin"]].append(line)
      
    return dataList

def parseAllReviews(path):
    file = open(path, 'r')
    dataList = defaultdict(lambda: [])
    for line in file:
        line = eval(line)
        dataList[line["asin"]].append(line)
      
    return dataList

print("Reading all reviews & all questions...")

allReviews = parseAllReviews("C:/Users/Moi/Downloads/reviews.json")
allQuestions = parseAllQueries("C:/Users/Moi/Downloads/qa.json")

#allReviews = parseAllReviews("/Users/Silvia/Downloads/reviews.json")
#allQuestions = parseAllQueries("/Users/Silvia/Downloads/qa.json")

# do we have to remove questions that have no reviews or reviews that have no questions??
docSet = []
for entry in allReviews.values():
    for review in entry:
        docSet.append(review["reviewText"])

for entry in allQuestions.values():
    for question in entry:
        docSet.append(question["question"])

docLen = [len(d.split()) for d in docSet]
avgdl = sum(docLen) / len(docLen)

print("done")

Reading all reviews & all questions...
done


In [79]:
def countAllWords():
    allWords = defaultdict(int)
    englishStopWords = stopwords.words('english')
    for r in allReviews.values():
        for review in r:
            review = review["reviewText"]
            exclude = set(string.punctuation)
            review = ''.join(ch for ch in review if ch not in exclude)
            for w in review.lower().split():
                if w not in englishStopWords:
                    allWords[w] += 1

    for q in allQuestions.values():
        for question in q:
            question = question["question"]
            exclude = set(string.punctuation)
            question = ''.join(ch for ch in question if ch not in exclude)
            for w in question.lower().split():
                if w not in englishStopWords:
                    allWords[w] += 1
    
    
    return allWords

allWords = countAllWords()

In [80]:
commonWords = sorted(allWords, key=lambda x: -allWords[x])[:5000]

In [81]:
def wordToIndex(term):
    if term in commonWords:
        return commonWords.index(term)
    else:
        return -1

In [82]:
def pairwiseProduct(review, question, length):
    reviewBag = [0]*length
    questionBag = [0]*length
    
    exclude = set(string.punctuation)
    review = ''.join(ch for ch in review if ch not in exclude)
    question = ''.join(ch for ch in question if ch not in exclude)
    
    for term in review.lower().split():
        index = wordToIndex(term)
        
        if index >= 0 and index < length:
            reviewBag[index] = review.lower().split().count(term)
        
    for term in question.lower().split():
        index = wordToIndex(term)
        
        if index >= 0 and index < length:
            questionBag[index] = question.lower().split().count(term)
        
    bagFeat = [0]*length
    
    for i in range(0, length):
        bagFeat[i] = reviewBag[i] * questionBag[i]
        
    #for i in range(0, length):
        #if reviewBag[i] > 0 or questionBag[i] > 0:
            #print(commonWords[i], reviewBag[i], questionBag[i], bagFeat[i])
        
    return bagFeat        

In [83]:
def featForOnlyQuestion(question, length):
    questionBag = [0]*length
    
    exclude = set(string.punctuation)
    question = ''.join(ch for ch in question if ch not in exclude)
        
    for term in question.lower().split():
        index = wordToIndex(term)
        
        if index >= 0 and index < length:
            questionBag[index] = question.lower().split().count(term)
        
    #for i in range(0, length):
        #if reviewBag[i] > 0 or questionBag[i] > 0:
            #print(commonWords[i], reviewBag[i], questionBag[i], bagFeat[i])
    
   
    return ([1]+questionBag)

In [84]:
def normalize(featList):
    
    max = 0
    min = float('inf')
    for feat in featList:
        if feat > max: max = feat
        if feat < min: min = feat        
    
    for i in range(0,len(featList)-1):
        if (max - min) == 0: 
            max = 1
            min = 0
        featList[i] = (featList[i] - min) / (max - min)

    return featList

In [98]:
def theLavFun(review, question):
    feat = [1]
    feat += pairwiseProduct(review, question, 1000)
    return feat

In [99]:
def train(X, y): 
    lr = LinearRegression()
    lr.fit(X, y)
    
    return lr


In [100]:
def test(y, y_hat):
    #print(sklearn.metrics.r2_score(y, y_hat))
    
    
    accuracy = sklearn.metrics.accuracy_score(y, y_hat)
    precision = sklearn.metrics.precision_score(y, y_hat)
    recall = sklearn.metrics.recall_score(y, y_hat)
    
    return "{0:.2f}, {1:.2f}, {2:.2f}".format(accuracy, precision, recall)

In [101]:
def constrain(elems, point):    
    for elem in elems:
        if (elem > point): yield 1
        else: yield 0

In [102]:
def generateFeature(data):
    for d in data:
        yield(theLavFun(d["review"], d["question"]))

In [103]:
X = list(generateFeature(data))
y = [1 if a == "Y" else 0 for a in answers]

In [104]:
random.seed(4410)

keys = list(range(1, len(answers)))
points = dict(zip(keys, zip(X, y)))
random.shuffle(keys)
X_rand = [points[key][0] for key in keys]
y_rand = [points[key][1] for key in keys]

X_train = X[:len(X_rand)*2//3]
y_train = y[:len(y_rand)*2//3]

X_test = X[len(X_rand)*2//3:]
y_test = y[len(y_rand)*2//3:]

In [105]:
lr = train(X_train, y_train)

In [73]:
y_hatTrain = lr.predict(X_train)
y_hatTest = lr.predict(X_test)

y_hatTrain = [1 for y_hat in y_hatTrain]
y_hatTest = [1 for y_hat in y_hatTest]

print("Predict Yes!")
print(test(y_train, y_hatTrain))
print(test(y_test, y_hatTest))


Predict Yes!
0.59, 0.59, 1.00
0.50, 0.50, 1.00


In [74]:
print("Predict with one question!")

y_hatTrain = lr.predict(X_train)
y_hatTest = lr.predict(X_test)

for cutoff in range(-20, 20):
    y_hatTrain_c = list(constrain(y_hatTrain, cutoff / 10))
    y_hatTest_c = list(constrain(y_hatTest, cutoff / 10))
    train_string = test(y_train, y_hatTrain_c)
    test_string = test(y_test, y_hatTest_c)

    print(cutoff / 10, ":\t", train_string, "\t", test_string)

print()



Predict with one question!
-2.0 :	 0.59, 0.59, 1.00 	 0.55, 0.53, 1.00
-1.9 :	 0.59, 0.59, 1.00 	 0.55, 0.53, 1.00
-1.8 :	 0.59, 0.59, 1.00 	 0.55, 0.53, 1.00
-1.7 :	 0.59, 0.59, 1.00 	 0.55, 0.53, 1.00
-1.6 :	 0.59, 0.59, 1.00 	 0.55, 0.53, 1.00
-1.5 :	 0.59, 0.59, 1.00 	 0.55, 0.53, 1.00
-1.4 :	 0.59, 0.59, 1.00 	 0.55, 0.53, 1.00
-1.3 :	 0.59, 0.59, 1.00 	 0.55, 0.53, 1.00
-1.2 :	 0.59, 0.59, 1.00 	 0.58, 0.55, 1.00
-1.1 :	 0.59, 0.59, 1.00 	 0.56, 0.53, 0.95
-1.0 :	 0.59, 0.59, 1.00 	 0.56, 0.53, 0.95
-0.9 :	 0.59, 0.59, 1.00 	 0.56, 0.53, 0.95
-0.8 :	 0.59, 0.59, 1.00 	 0.56, 0.53, 0.95
-0.7 :	 0.59, 0.59, 1.00 	 0.56, 0.53, 0.95
-0.6 :	 0.59, 0.59, 1.00 	 0.59, 0.55, 0.95
-0.5 :	 0.59, 0.59, 1.00 	 0.62, 0.57, 0.95
-0.4 :	 0.59, 0.59, 1.00 	 0.62, 0.57, 0.95
-0.3 :	 0.59, 0.59, 1.00 	 0.64, 0.58, 0.95
-0.2 :	 0.59, 0.59, 1.00 	 0.64, 0.58, 0.95
-0.1 :	 0.59, 0.59, 1.00 	 0.64, 0.58, 0.95
0.0 :	 0.73, 0.69, 1.00 	 0.61, 0.57, 0.86
0.1 :	 1.00, 1.00, 1.00 	 0.70, 0.65, 0.86
0.2 :	 

c:\python36-64\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [106]:
print("Predict with one question and the most relevant review!")

y_hatTrain = lr.predict(X_train)
y_hatTest = lr.predict(X_test)

for cutoff in range(-20, 20):
    y_hatTrain_c = list(constrain(y_hatTrain, cutoff / 10))
    y_hatTest_c = list(constrain(y_hatTest, cutoff / 10))
    train_string = test(y_train, y_hatTrain_c)
    test_string = test(y_test, y_hatTest_c)

    print(cutoff / 10, ":\t", train_string, "\t", test_string)

print()

Predict with one question and the most relevant review!
-2.0 :	 0.69, 0.69, 1.00 	 0.61, 0.64, 0.74
-1.9 :	 0.69, 0.69, 1.00 	 0.61, 0.64, 0.74
-1.8 :	 0.69, 0.69, 1.00 	 0.61, 0.64, 0.74
-1.7 :	 0.69, 0.69, 1.00 	 0.61, 0.64, 0.74
-1.6 :	 0.69, 0.69, 1.00 	 0.61, 0.64, 0.74
-1.5 :	 0.69, 0.69, 1.00 	 0.61, 0.64, 0.74
-1.4 :	 0.69, 0.69, 1.00 	 0.59, 0.64, 0.72
-1.3 :	 0.69, 0.69, 1.00 	 0.59, 0.64, 0.72
-1.2 :	 0.69, 0.69, 1.00 	 0.59, 0.64, 0.72
-1.1 :	 0.69, 0.69, 1.00 	 0.59, 0.64, 0.72
-1.0 :	 0.69, 0.69, 1.00 	 0.59, 0.64, 0.72
-0.9 :	 0.69, 0.69, 1.00 	 0.59, 0.64, 0.72
-0.8 :	 0.69, 0.69, 1.00 	 0.59, 0.64, 0.72
-0.7 :	 0.69, 0.69, 1.00 	 0.59, 0.64, 0.72
-0.6 :	 0.69, 0.69, 1.00 	 0.59, 0.64, 0.72
-0.5 :	 0.69, 0.69, 1.00 	 0.59, 0.64, 0.72
-0.4 :	 0.69, 0.69, 1.00 	 0.61, 0.65, 0.72
-0.3 :	 0.69, 0.69, 1.00 	 0.61, 0.65, 0.72
-0.2 :	 0.69, 0.69, 1.00 	 0.61, 0.65, 0.72
-0.1 :	 0.70, 0.70, 1.00 	 0.61, 0.65, 0.72
0.0 :	 0.82, 0.79, 1.00 	 0.61, 0.65, 0.72
0.1 :	 0.91, 0.88, 1.

c:\python36-64\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
